# EDA/ETL Prototype

In this notebook we prototype the data acquisition and cleaning process. First we import all the necessary libraries.

In [ ]:
from netCDF4 import Dataset
import numpy as np
from datetime import datetime, timedelta
from dateutil import rrule
import pandas as pd
from scipy.interpolate import RegularGridInterpolator
import requests

The first dataset that is acquired is the turbine metadata. This is returned from the API as a JSON object, which we convert to the Pandas DataFrame.

In [ ]:
r = requests.get("https://eersc.usgs.gov/api/uswtdb/v1/turbines?&t_state=in.(WA,OR)&select=t_cap,t_hh,xlong,ylat")
turbine_data = pd.DataFrame(r.json()).dropna()
turbine_data["xlong"] = 360. + turbine_data["xlong"]
turbine_data.to_csv("turbine_metadata.csv")

Next we download the wind power data from the BPA Balancing Authority API. This file is a CSV file, which we read into a Pandas DataFrame.

In [ ]:
power_data = pd.read_csv("https://transmission.bpa.gov/Business/Operations/Wind/twndbspt.txt", skiprows=10, delimiter="\t")
power_data.set_index(power_data.columns[0], inplace=True)
power_data.index = pd.to_datetime(power_data.index) + timedelta(hours=8)

We define the latitude and longitude ranges of the Pacific Northwest.

In [ ]:
LON_MIN = 360. - 125.
LON_MAX = 360. - 115.
LAT_MIN = 40.
LAT_MAX = 50.

Here we write a function that downloads the weather simulation data from the NOAA API. The wind speed is calculated from the velocity, and the temperate is also extracted. The function downloads data for every hour for a single specified day.

In [ ]:
def get_vel_data_for_date(date, LON_MIN, LON_MAX, LAT_MIN, LAT_MAX):
    date_str = date.strftime("%Y%m%d")
    print("Downloading weather simulation data for ", date_str)
    
    vel = []
    T = []
    
    hs_vel = np.array([10, 20, 30, 40, 50, 80, 100])
    hs_T = np.array([2, 80, 100])
    
    for hr_base in [0, 6, 12, 18]:
        try:
            weather_data_url = "https://nomads.ncep.noaa.gov/dods/gfs_0p25_1hr/gfs{}/gfs_0p25_1hr_{:02d}z".format(date_str, hr_base)
            weather_data = Dataset(weather_data_url)
            hrs = range(6)
        except OSError as e:
            hr_base -= 6
            weather_data_url = "https://nomads.ncep.noaa.gov/dods/gfs_0p25_1hr/gfs{}/gfs_0p25_1hr_{:02d}z".format(date_str, hr_base)
            weather_data = Dataset(weather_data_url)
            hrs = range(6, 12)
        
        lon = weather_data.variables["lon"][:]
        lat = weather_data.variables["lat"][:]

        pnw_lat_idx = np.logical_and(LAT_MIN <= lat, lat <= LAT_MAX)
        pnw_lon_idx = np.logical_and(LON_MIN <= lon, lon <= LON_MAX)

        for hr in hrs:
            vel_hr = np.zeros((np.sum(pnw_lat_idx), np.sum(pnw_lon_idx), len(hs_vel)))
            for k, h_vel in enumerate(hs_vel):
                vel_hr[:,:,k] = np.sqrt(weather_data.variables["ugrd{}m".format(h_vel)][hr,:,:]**2 \
                                      + weather_data.variables["vgrd{}m".format(h_vel)][hr,:,:]**2)[np.ix_(pnw_lat_idx, pnw_lon_idx)]
            vel.append(vel_hr)
            
            T_hr = np.zeros((np.sum(pnw_lat_idx), np.sum(pnw_lon_idx), len(hs_T)))
            for k, h_T in enumerate(hs_T):
                T_hr[:,:,k] = weather_data.variables["tmp{}m".format(h_T)][hr,:,:][np.ix_(pnw_lat_idx, pnw_lon_idx)]
            T.append(T_hr)

    return vel, T, lat[pnw_lat_idx], lon[pnw_lon_idx], hs_vel, hs_T

Here we write a function that takes wind data (or temperature data) at a specific time, and interpolates it to the turbine locations using trilinear interpolation.

In [ ]:
def interpolate_wind_data(turbine_data, vel, lat, lon, hs):
    rgi = RegularGridInterpolator((lat, lon, hs,), vel, method="linear")
    turb_locs = turbine_data[["ylat", "xlong", "t_hh"]].to_numpy()
    return rgi(turb_locs)

Finally, we put it all together into a single loop that downloads data for the past few days, interpolates the weather data to the turbine locations, and then combines the data with the wind power generation data and saves it to a CSV file.

In [ ]:
data = pd.DataFrame(columns=["turb_{}_vel".format(i) for i in range(len(turbine_data))] + ["turb_{}_T".format(i) for i in range(len(turbine_data))])

num_days = 4
for date_time in rrule.rrule(rrule.DAILY, dtstart=datetime.today().date() - timedelta(days=num_days + 1), until=datetime.today().date()):
    vel, T, lat, lon, hs_vel, hs_T = get_vel_data_for_date(date_time, LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)
    for hr in range(24):
        data.loc[data.shape[0]] = np.hstack((interpolate_wind_data(turbine_data, vel[hr], lat, lon, hs_vel),
                                             interpolate_wind_data(turbine_data, T[hr], lat, lon, hs_T)))
        
data = data.set_index(pd.date_range(datetime.today().date() - timedelta(days=num_days + 1), datetime.today().date() + timedelta(days=1), freq="H", closed="left"))
data = data.join(power_data)
data.to_csv("sample_data.csv")

In the full application, these functions will be rewritten so that the data is properly stored in a database and incrementally updated as it becomes available.